In [166]:
import pandas as pd
import numpy as np
#Final project for Data Science Certification on Coursera
print("Hello Capstone Project Course!")

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Hello Capstone Project Course!
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [168]:
#Use pandas read_html to grab the first table and put it in dataframe 'hood'
hood = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)[0]
hood.columns = (['Postal Code','Borough','Neighborhood'])
print('Raw Frame')
print(hood.shape)
print(hood.head(10))
#Drop the unassigned borough rows
hood = hood[hood.Borough != 'Not assigned']
print('Dropped unassigned boroughs')
print(hood.shape)
print(hood.head(10))
#Convert neighborhoods to borough where the neighborhood is unassigned
hood['Neighborhood'][hood.Neighborhood == 'Not assigned'] = hood.Borough
print('Convert unassigned neighborhoods to borough name')
print(hood.shape)
print(hood.head(10))
#Group the PostalCode and Borough columns
hood = hood.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
print('Group by Postal and Borough and combine neighborhoods')
print(hood.shape)
print(hood.head())

Raw Frame
(288, 3)
  Postal Code           Borough      Neighborhood
0         M1A      Not assigned      Not assigned
1         M2A      Not assigned      Not assigned
2         M3A        North York         Parkwoods
3         M4A        North York  Victoria Village
4         M5A  Downtown Toronto      Harbourfront
5         M5A  Downtown Toronto       Regent Park
6         M6A        North York  Lawrence Heights
7         M6A        North York    Lawrence Manor
8         M7A      Queen's Park      Not assigned
9         M8A      Not assigned      Not assigned
Dropped unassigned boroughs
(211, 3)
   Postal Code           Borough      Neighborhood
2          M3A        North York         Parkwoods
3          M4A        North York  Victoria Village
4          M5A  Downtown Toronto      Harbourfront
5          M5A  Downtown Toronto       Regent Park
6          M6A        North York  Lawrence Heights
7          M6A        North York    Lawrence Manor
8          M7A      Queen's Park     

In [169]:
#The geocoder package did not work so grab lat/lon from the csv file
geocodes = pd.read_csv('http://cocl.us/Geospatial_data')
geocodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [170]:
#Merge the hood dataframe with the geocodes dataframe on Postal Code
hood_coded = hood.merge(geocodes, left_on='Postal Code', right_on='Postal Code')
hood_coded.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [171]:
#Use geolocator to get the coordinates of Toronto
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
tor_lat = location.latitude
tor_lon = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(tor_lat, tor_lon))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [172]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[tor_lat, tor_lon], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(hood_coded['Latitude'], hood_coded['Longitude'], hood_coded['Borough'], hood_coded['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [173]:
#FourSquare credentials
CLIENT_ID = 'P55ZSLKQNZJYISSRDA1DAD1VONS5ZEFMO05L3AWTYED5YXSL' # your Foursquare ID
CLIENT_SECRET = 'UCNJEBTYKKQ3ZCFN3H3DJXJZUK1FVHNIOQNQYR3PLQAS1LPU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P55ZSLKQNZJYISSRDA1DAD1VONS5ZEFMO05L3AWTYED5YXSL
CLIENT_SECRET:UCNJEBTYKKQ3ZCFN3H3DJXJZUK1FVHNIOQNQYR3PLQAS1LPU


In [174]:
hood_coded.loc[0, 'Neighborhood']

'Rouge, Malvern'

In [175]:
#Neighborhood lat/lon
neighborhood_latitude = hood_coded.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = hood_coded.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = hood_coded.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


In [176]:
#Top 100 Venues within 1km of neighborhood
radius = 1000
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    VERSION, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cb8f3526a60715addaa1f79'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4d669cba83865481c948fa53-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ed941735',
         'name': 'Spa',
         'pluralName': 'Spas',
         'primary': True,
         'shortName': 'Spa'}],
       'id': '4d669cba83865481c948fa53',
       'location': {'address': '8130 Sheppard Ave E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside Ave',
        'distance': 595,
        'formattedAddress': ['8130 Sheppard Ave E (Morningside Ave)',
         'Toronto ON M1B 3W3',
         'Canada'],
        'labeledLatLngs': [{'label': 'd

In [177]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [178]:
#Clean the json structure and put it in a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
2,Harvey's,Fast Food Restaurant,43.800106,-79.198258
3,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
4,Wendy's,Fast Food Restaurant,43.802008,-79.198080


In [179]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


In [180]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
# Use the above function

toronto_venues = getNearbyVenues(names=hood_coded['Neighborhood'],
                                   latitudes=hood_coded['Latitude'],
                                   longitudes=hood_coded['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [181]:
print(toronto_venues.shape)
toronto_venues.head()

(2247, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [182]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,56,56,56,56,56,56


In [183]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


In [184]:
#Analyze each neighborhood
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [185]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
1,Agincourt,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [186]:
toronto_grouped.shape

(100, 277)

In [187]:
#Top 5 venues per neighborhood
num_top_venues = 5

for thood in toronto_grouped['Neighborhood']:
    print("----"+thood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == thood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1           Steakhouse  0.04
2                 Café  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Agincourt----
            venue  freq
0    Skating Rink  0.25
1          Lounge  0.25
2  Breakfast Spot  0.25
3  Sandwich Place  0.25
4  Medical Center  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
              venue  freq
0        Playground  0.33
1              Park  0.33
2  Sculpture Garden  0.33
3       Yoga Studio  0.00
4     Metro Station  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
            venue  freq
0     Pizza Place   0.2
1   Grocery Store   0.2
2  Sandwich Place   0.1
3        Pharmacy   0.1
4    Liquor Store   0.1


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place   0.2
1        Pharmacy   0.1
2     Coffee Shop   0.1
3    Skating 

In [188]:
#Sort Function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [189]:
#Putting it all in a dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,American Restaurant,Steakhouse,Café,Thai Restaurant,Asian Restaurant,Restaurant,Burger Joint,Bar,Bakery
1,Agincourt,Lounge,Sandwich Place,Skating Rink,Breakfast Spot,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Sculpture Garden,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Pharmacy,Sandwich Place,Beer Store,Fast Food Restaurant,Liquor Store,General Travel,General Entertainment
4,"Alderwood, Long Branch",Pizza Place,Gym,Athletics & Sports,Pharmacy,Pool,Pub,Sandwich Place,Skating Rink,Coffee Shop,Gay Bar


In [190]:
#Cluster the neighborhoods
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 4, 1, 2, 2, 2, 2, 2, 2], dtype=int32)

In [191]:
#New dataframe with clusters and venues
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster', kmeans.labels_)

toronto_merged = hood_coded

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2.0,Bar,Moving Target,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2.0,Electronics Store,Medical Center,Mexican Restaurant,Spa,Pizza Place,Rental Car Location,Intersection,Breakfast Spot,Dim Sum Restaurant,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Hakka Restaurant,Fried Chicken Joint,Lounge,Thai Restaurant,Bakery,Caribbean Restaurant,Bank,Athletics & Sports,Donut Shop,Doner Restaurant


In [195]:
#Fix the Cluster label for colors - needs to be int
toronto_merged.dropna(inplace = True)
toronto_merged['Cluster'] = toronto_merged['Cluster'].astype(np.int64)
toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,Bar,Moving Target,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Electronics Store,Medical Center,Mexican Restaurant,Spa,Pizza Place,Rental Car Location,Intersection,Breakfast Spot,Dim Sum Restaurant,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Hakka Restaurant,Fried Chicken Joint,Lounge,Thai Restaurant,Bakery,Caribbean Restaurant,Bank,Athletics & Sports,Donut Shop,Doner Restaurant


In [200]:
# create map to visualize the clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [201]:
#Analyze first cluster
toronto_merged.loc[toronto_merged['Cluster'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,"Humber Bay, King's Mill Park, Kingsway Park So...",0,Baseball Field,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Department Store
97,"Emery, Humberlea",0,Baseball Field,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Department Store


In [202]:
#Analyze second cluster
toronto_merged.loc[toronto_merged['Cluster'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",1,Fast Food Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
15,L'Amoreaux West,1,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Breakfast Spot,Indian Restaurant,Electronics Store,Discount Store,Cosmetics Shop,Pharmacy,Pizza Place
25,Parkwoods,1,Park,Fast Food Restaurant,Pool,Food & Drink Shop,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
31,Downsview West,1,Grocery Store,Moving Target,Shopping Mall,Bank,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
33,Downsview Northwest,1,Gym / Fitness Center,Grocery Store,Liquor Store,Discount Store,Athletics & Sports,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
81,"The Junction North, Runnymede",1,Grocery Store,Pizza Place,Brewery,Caribbean Restaurant,Convenience Store,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner
92,"Kingsway Park South West, Mimico NW, The Queen...",1,Grocery Store,Convenience Store,Fast Food Restaurant,Burger Joint,Sandwich Place,Discount Store,Bakery,Supplement Shop,Tanning Salon,Wings Joint
101,"Albion Gardens, Beaumond Heights, Humbergate, ...",1,Grocery Store,Pizza Place,Fried Chicken Joint,Pharmacy,Sandwich Place,Beer Store,Fast Food Restaurant,Liquor Store,General Travel,General Entertainment


In [203]:
#Analyze third cluster
toronto_merged.loc[toronto_merged['Cluster'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",2,Bar,Moving Target,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
2,"Guildwood, Morningside, West Hill",2,Electronics Store,Medical Center,Mexican Restaurant,Spa,Pizza Place,Rental Car Location,Intersection,Breakfast Spot,Dim Sum Restaurant,Diner
3,Woburn,2,Coffee Shop,Korean Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
4,Cedarbrae,2,Hakka Restaurant,Fried Chicken Joint,Lounge,Thai Restaurant,Bakery,Caribbean Restaurant,Bank,Athletics & Sports,Donut Shop,Doner Restaurant
5,Scarborough Village,2,Convenience Store,Playground,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
6,"East Birchmount Park, Ionview, Kennedy Park",2,Discount Store,Convenience Store,Bus Station,Coffee Shop,Department Store,Donut Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
7,"Clairlea, Golden Mile, Oakridge",2,Bus Line,Bakery,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Metro Station,Dog Run,Diner,Discount Store
8,"Cliffcrest, Cliffside, Scarborough Village West",2,Motel,Skating Rink,American Restaurant,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
9,"Birch Cliff, Cliffside West",2,College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
10,"Dorset Park, Scarborough Town Centre, Wexford ...",2,Indian Restaurant,Chinese Restaurant,Light Rail Station,Latin American Restaurant,Furniture / Home Store,Pet Store,Vietnamese Restaurant,Comfort Food Restaurant,Department Store,Electronics Store


In [204]:
#Analyze fourth cluster
toronto_merged.loc[toronto_merged['Cluster'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,Humber Summit,3,Pizza Place,Empanada Restaurant,Dog Run,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
100,"Kingsview Village, Martin Grove Gardens, Richv...",3,Pizza Place,Mobile Phone Shop,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant


In [205]:
#Anayze fifth cluster
toronto_merged.loc[toronto_merged['Cluster'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North, L'Amoreaux East, Milliken, St...",4,Park,Playground,Sculpture Garden,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
20,"Silver Hills, York Mills",4,Cafeteria,Park,Martial Arts Dojo,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
23,York Mills West,4,Park,Bank,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
30,"CFB Toronto, Downsview East",4,Park,Bus Stop,Airport,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
40,East Toronto,4,Park,Convenience Store,Coffee Shop,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
44,Lawrence Park,4,Park,Bus Line,Swim School,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Deli / Bodega
50,Rosedale,4,Park,Trail,Playground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
64,"Forest Hill North, Forest Hill West",4,Park,Trail,Sushi Restaurant,Jewelry Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
74,Caledonia-Fairbanks,4,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Airport,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant
79,"Downsview, North Park, Upwood Park",4,Basketball Court,Park,Bakery,Construction & Landscaping,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


In [206]:
#Create most common venue lists for each cluster

tor1 = toronto_merged.loc[toronto_merged['Cluster'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
tor1_1st = tor1[['Neighborhood','1st Most Common Venue']]
tor1_1st

tor2 = toronto_merged.loc[toronto_merged['Cluster'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
tor2_1st = tor2[['Neighborhood','1st Most Common Venue']]
tor2_1st

tor3 = toronto_merged.loc[toronto_merged['Cluster'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
tor3_1st = tor3[['Neighborhood','1st Most Common Venue']]
tor3_1st

tor4 = toronto_merged.loc[toronto_merged['Cluster'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
tor4_1st = tor4[['Neighborhood','1st Most Common Venue']]
tor4_1st

tor5 = toronto_merged.loc[toronto_merged['Cluster'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
tor5_1st = tor5[['Neighborhood','1st Most Common Venue']]
tor5_1st

,Neighborhood,1st Most Common Venue
14,"Agincourt North, L'Amoreaux East, Milliken, St...",Park
20,"Silver Hills, York Mills",Cafeteria
23,York Mills West,Park
30,"CFB Toronto, Downsview East",Park
40,East Toronto,Park
44,Lawrence Park,Park
50,Rosedale,Park
64,"Forest Hill North, Forest Hill West",Park
74,Caledonia-Fairbanks,Park
79,"Downsview, North Park, Upwood Park",Basketball Court


In [207]:
#Print the most common venue for each cluster

tor1_name = tor1_1st.groupby(['1st Most Common Venue'])['Neighborhood'].count()
tor1_name = tor1_name.sort_values(axis =0,ascending = False)
print('Toronto Group 1')
print(tor1_name.head(1))
print()

tor2_name = tor2_1st.groupby(['1st Most Common Venue'])['Neighborhood'].count()
tor2_name = tor2_name.sort_values(axis =0,ascending = False)
print('Toronto Group 2')
print(tor2_name.head(1))
print()

tor3_name = tor3_1st.groupby(['1st Most Common Venue'])['Neighborhood'].count()
tor3_name = tor3_name.sort_values(axis =0,ascending = False)
print('Toronto Group 3')
print(tor3_name.head(1))
print()

tor4_name = tor4_1st.groupby(['1st Most Common Venue'])['Neighborhood'].count()
tor4_name = tor4_name.sort_values(axis =0,ascending = False)
print('Toronto Group 4')
print(tor4_name.head(1))
print()

tor5_name = tor5_1st.groupby(['1st Most Common Venue'])['Neighborhood'].count()
tor5_name = tor5_name.sort_values(axis =0,ascending = False)
print('Toronto Group 5')
print(tor5_name.head(1))


Toronto Group 1
1st Most Common Venue
Baseball Field    2
Name: Neighborhood, dtype: int64

Toronto Group 2
1st Most Common Venue
Grocery Store    4
Name: Neighborhood, dtype: int64

Toronto Group 3
1st Most Common Venue
Coffee Shop    23
Name: Neighborhood, dtype: int64

Toronto Group 4
1st Most Common Venue
Pizza Place    2
Name: Neighborhood, dtype: int64

Toronto Group 5
1st Most Common Venue
Park    9
Name: Neighborhood, dtype: int64
